In [1]:
# %pip install 'networkx<2.7'

In [2]:
# !pip install cdlib

In [3]:
import os
import heapq
import argparse
import threading
import multiprocessing
import sys
import queue


from __future__ import division
import math, time
from copy import deepcopy
import multiprocessing, json
from tqdm import tqdm
import heapq


#importing libraries that will be used
# import networkx as nx#for creating network
# import numpy as np
# import matplotlib.pyplot as plt#for plotting plots
# import random
# import time#claculating time
# import math
# from collections import Counter
# from itertools import permutations 
# from itertools import combinations

# from collections import defaultdict
# from scipy.io import mmread# to read dataset
# import pandas as pd



In [4]:
#importing libraries that will be used
import networkx as nx#for creating network
import numpy as np
import matplotlib.pyplot as plt#for plotting plots

import random
import time#claculating time
import math
from collections import Counter
from itertools import permutations 
from itertools import combinations

from collections import defaultdict
from scipy.io import mmread# to read dataset
import pandas as pd

# from cdlib import algorithms

In [5]:
def celf(graph, k):
    """
    Implementation of CELF algorithm for influence maximization in social networks
    
    Args:
    - graph: NetworkX graph object representing the social network
    - k: number of nodes to select
    
    Returns:
    - nodes: list of k nodes with the highest influence scores
    """
    # Initialize variables
    nodes = []
    heap = []
    marg_gains = {}

    # Calculate the marginal gain for each node
    for node in graph.nodes():
        # Run Monte Carlo simulations to estimate the influence of each node
        sim_res = linear_Threshold(graph, nodes + [node])
        marg_gains[node] = len(sim_res) - len(nodes)
        # Add the node to the heap with its marginal gain as key
        heapq.heappush(heap, (-marg_gains[node], node))

    # Select the k nodes with the highest influence scores
    while len(nodes) < k:
        # Get the node with the highest marginal gain
        _, node = heapq.heappop(heap)
        # Recalculate the marginal gain of the selected node
        sim_res = linear_Threshold(graph, nodes + [node])
        marg_gains[node] = len(sim_res) - len(nodes)
        # Add the node to the list of selected nodes
        nodes.append(node)
        # Update the heap with the new marginal gains
        for n in graph.neighbors(node):
            if n not in nodes:
                heapq.heappush(heap, (-marg_gains[n], n))

    return nodes


In [6]:
'''Implementation of SimPath algorithm'''

class CELFQueue:
    # create if not exist
    nodes = None
    q = None
    nodes_gain = None

    def __init__(self):
        self.q = []
        self.nodes_gain = {}

    def put(self, node, marginalgain):
        self.nodes_gain[node] = marginalgain
        heapq.heappush(self.q, (-marginalgain, node))

    def update(self, node, marginalgain):
        self.remove(node)
        self.put(node, marginalgain)

    def remove(self, node):
        self.q.remove((-self.nodes_gain[node], node))
        self.nodes_gain[node] = None
        heapq.heapify(self.q)

    def topn(self, n):
        top = heapq.nsmallest(n, self.q)
        top_ = list()
        for t in top:
            top_.append(t[1])
        return top_

    def get_gain(self, node):
        return self.nodes_gain[node]

    
    
def init_D(graph):
    D = {}
#     for i in range(graph.node_num + 1):
    for i in graph.nodes:
        D[i]=[]
    return D

class Graph:
    nodes = None
    edges = None
    children = None
    parents = None
    node_num = None
    edge_num = None

    def __init__(self, nodes, edges, children, parents, node_num, edge_num):
#         print("hello[[[[[[[[[[[[[[[[[[[[[]]]]]]]]]]]]]]]]]]]]]")
        self.nodes = nodes
        self.edges = edges
        self.children = children
        self.parents = parents
        self.node_num = node_num
        self.edge_num = edge_num
#         print("Numnodes:",self.node_num)
    def get_children(self, node):
        ch = self.children.get(node)
        if ch is None:
            self.children[node] = []
        return self.children[node]

    def get_parents(self, node):
        pa = self.parents.get(node)
        if pa is None:
            self.parents[node] = []
        return self.parents[node]

    def get_weight(self, src, dest):
        weight = self.edges.get((src, dest))
        if weight is None:
            return 0
        else:
            return weight

    # return true if node1 is parent of node 2 , else return false
    def is_parent_of(self, node1, node2):
        if self.get_weight(node1, node2) != 0:
            return True
        else:
            return False

    # return true if node1 is child of node 2 , else return false
    def is_child_of(self, node1, node2):
        return self.is_parent_of(node2, node1)

    def get_out_degree(self, node):
        return len(self.get_children(node))

    def get_in_degree(self, node):
        return len(self.get_parents(node))

    
def read_graph_info(path):
    if os.path.exists(path):
        parents = {}
        children = {}
        edges = {}
        nodes = set()

        try:
            f = open(path, 'r')
            txt = f.readlines()
            header = str.split(txt[0])
            node_num = int(header[0])
            edge_num = int(header[1])

            for line in txt[1:]:
#                 print("line:",line)
                row = str.split(line)
#                 print(row)
                src = int(row[0])
                des = int(row[1])
                nodes.add(src)
                nodes.add(des)
#                 print(src,des)    
                if children.get(src) is None:
                    children[src] = []
                if parents.get(des) is None:
                    parents[des] = []

#                 weight = float(row[2])
                weight=round(random.uniform(0.0,1.0),2)
                edges[(src, des)] = weight
                children[src].append(des)
                parents[des].append(src)

            return list(nodes), edges, children, parents, node_num, edge_num
        except IOError:
            print('IOError')
    else:
        print('file can not found')

        
        

# def get_vertex_cover(graph):
#     # dv[i] out degree of node i+1
#     dv = np.zeros(graph.node_num)
#     # e[i,j] = 0: edge (i+1,j+1),(j+1,i+1) checked
#     check_array = np.zeros((graph.node_num, graph.node_num))
#     checked = 0
# #     print(range(graph.node_num))
#     for i in range(graph.node_num):
#         # for a edge (i,j) and (j,i) may be count twice but the algorithm is to find a vertex cover. it doesn't mater
#         dv[i] = graph.get_out_degree(i + 1) + graph.get_in_degree(i + 1)
#     # V: Vertex cover
#     V = set()
# #     print(dv)
#     while checked < graph.edge_num:
#         s = dv.argmax() + 1
#         V.add(s)
#         # make sure that never to select this node again
#         children = graph.get_children(s)
#         parents = graph.get_parents(s)
#         for child in children:
#             if check_array[s - 1][child - 1] == 0:
#                 check_array[s - 1][child - 1] = 1
#                 checked = checked + 1
#         for parent in parents:
#             if check_array[parent - 1][s - 1] == 0:
#                 check_array[parent - 1][s - 1] = 1
#                 checked = checked + 1
#         dv[s - 1] = -1
#     return list(V)



def forward(Q, D, spd, pp, r, W, U, spdW_u, graph):
    x = Q[-1]
    if U is None:
        U = []
    children = graph.get_children(x)
    count = 0
    while True:
        # any suitable chid is ok

#         for child in range(count, len(children)):
        flag=1
        for y in children:
            
#             print("y,D[x],type:",y,D[x],type(D[x]))
#             if(y not in D[x]):
#                 print("YES")
                
            if (y in W) and (y not in Q) and (y not in D[x]):
#                 y = children[child]
                flag=0
                break
#             count = count + 1

        # no such child:
        if flag==1:
            return Q, D, spd, pp

        if pp * graph.get_weight(x, y) < r:
            D[x].append(y)
        else:
            Q.append(y)
            pp = pp * graph.get_weight(x, y)
            spd = spd + pp
            D[x].append(y)
            x = Q[-1]
            for v in U:
                if v not in Q:
                    spdW_u[v] = spdW_u[v] + pp
            children = graph.get_children(x)
            count = 0


            
def backtrack(u, r, W, U, spdW_, graph):
    Q = [u]
    spd = 1
    pp = 1
    D = init_D(graph)

    while len(Q) != 0:
        Q, D, spd, pp = forward(Q, D, spd, pp, r, W, U, spdW_, graph)
        u = Q.pop()
#         print("In backtrack:type,Q,u",type(Q),Q,u)
        D[u] = []
        if len(Q) != 0:
            v = Q[-1]
            pp = pp / graph.get_weight(v, u)
    return spd



def simpath_spread(S, r, U, graph, spdW_=None):
    spread = 0
    # W: V-S
    W = set(graph.nodes).difference(S)
    if U is None or spdW_ is None:
        spdW_={}
        for i in graph.nodes:
            spdW_[i]=0
#         spdW_ = np.zeros(graph.node_num + 1)
        # print 'U None'
    for u in S:
        W.add(u)
        # print spdW_[u]
        spread = spread + backtrack(u, r, W, U, spdW_[u], graph)
        # print spdW_[u]
        W.remove(u)
    return spread





def simpath(graph, k, r, l):
    C = set(get_vertex_cover(graph))
    V = set(graph.nodes)

    V_C = V.difference(C)
    # spread[x] is spd of S + x
#     spread = np.zeros(graph.node_num + 1)
    spread={}
    for i in graph.nodes:
        spread[i]=0
#     spdV_ = np.ones((graph.node_num + 1, graph.node_num + 1))
    spdV_={}
    for i in graph.nodes:
        dd={}
        for j in graph.nodes:
            dd[j]=0
        spdV_[i]=dd
        
        
    for u in C:
        U = V_C.intersection(set(graph.get_parents(u)))
        spread[u] = simpath_spread(set([u]), r, U, graph, spdV_)
    for v in V_C:
        v_children = graph.get_children(v)
        for child in v_children:
            spread[v] = spread[v] + spdV_[child][v] * graph.get_weight(v, child)
        spread[v] = spread[v] + 1
    celf = CELFQueue()
    # put all nodes into celf queqe
    # spread[v] is the marginal gain at this time
    
#     for node in range(1, graph.node_num + 1):
#         celf.put(node, spread[node])
    for node in graph.nodes:
        celf.put(node, spread[node])
    
    S = set()
    W = V
    spd = 0
    # mark the node that checked before during the same Si
#     checked = np.zeros(graph.node_num + 1)
    checked={}
    for i in graph.nodes:
        checked[i]=0

    while len(S) < k:
        U = celf.topn(l)
#         spdW_ = np.ones((graph.node_num + 1, graph.node_num + 1))
#         spdV_x = np.zeros(graph.node_num + 1)
        
        spdW_={}
        spdV_x={}
        for i in graph.nodes:
            dd={}
            spdV_x[i]=0
            for j in graph.nodes:
                dd[j]=1
            spdW_[i]=dd
        
        simpath_spread(S, r, U, graph, spdW_=spdW_)
        for x in U:
            for s in S:
                spdV_x[x] = spdV_x[x] + spdW_[s][x]
        for x in U:
            if checked[x] != 0:
                S.add(x)
                W = W.difference(set([x]))
                spd = spread[x]
                # print spread[x],simpath_spread(S,r,None,None)
#                 checked = np.zeros(graph.node_num + 1)
#                 for i in graph.nodes:
#                     checked[i]=0
                celf.remove(x)
                break
            else:
                spread[x] = backtrack(x, r, W, None, None, graph) + spdV_x[x]
                checked[x] = 1
                celf.update(x, spread[x] - spd)
    return S



# node,edges,children,parents,nodenum,edge_num=read_graph_info('dolphins.txt') #Input dataset with 'total nodes' and 'total edges' in the 'first line'
# graph = Graph(node,edges,children,parents,nodenum,edge_num)
# seeds = simpath(graph, 3, 0.5, 4)
# print("seed:",seeds)



def SIMPATH_setup(G):
    parents = {}
    children = {}
    edges = {}
    nodes = set()
    node_num = len(G.nodes())
    edge_num = len(G.edges())
#     print(node_num,edge_num)
    for src,des in G.edges():
        nodes.add(src)
        nodes.add(des)
        if children.get(src) is None:
            children[src] = []
        if parents.get(des) is None:
            parents[des] = []
        weight=G[src][des]['weight']
        edges[(src, des)] = weight
        children[src].append(des)
        parents[des].append(src)
    return list(nodes), edges, children, parents, node_num, edge_num
     
def run_SIMPATH(G,k):
    node,edges,children,parents,nodenum,edge_num=SIMPATH_setup(G)
    graph = Graph(node,edges,children,parents,nodenum,edge_num)
    seeds = simpath(graph, k, 0.5, k+1)
    return list(seeds)




def get_vertex_cover(graph):
#     dv = np.zeros(graph.node_num)
    dv=dict()
#     check_array = np.zeros((graph.node_num, graph.node_num))
    check_array={}
    for i in graph.nodes:
        dd={}
        for j in graph.nodes:
            dd[j]=0
        check_array[i]=dd
        
#     checked = 0
    
#     for i in range(graph.node_num):
#         dv[i] =graph.get_out_degree(i + 1) + graph.get_in_degree(i + 1)
    for i in graph.nodes:
        dv[i]=graph.get_out_degree(i) + graph.get_in_degree(i)
    
    # V: Vertex cover
    V = set()
#     while checked < graph.edge_num:
    for checked in graph.nodes:    
#         s = dv.argmax() + 1
        _,s=max(zip(dv.values(), dv.keys()))
        V.add(s)
        # make sure that never to select this node again
        children = graph.get_children(s)
        parents = graph.get_parents(s)
        for child in children:
            if check_array[s][child] == 0:
                check_array[s][child] = 1
#                 checked = checked + 1
        for parent in parents:
            if check_array[parent][s] == 0:
                check_array[parent][s] = 1
#                 checked = checked + 1
        dv[s] = -1
#     print("In Vertex conver V is:",V)
    return list(V)


In [7]:
# -- coding: utf-8 --
# @Time : 2022/5/14 16:00
# @Author : Mkc
# @Email : mkc17@foxmail.com
# @Software: PyCharm

import numpy as np
import networkx as nx
import time
import random
import math
import copy
from time import strftime, localtime

def IC_model(graph,seeds,mc,p=0.01):
    g=graph
    S=seeds
    spread = []
    for i in range(mc):
        new_active, Au = S[:], S[:]
        while new_active:
            new_ones = []
            for node in new_active:
                nbs = list(set(g.neighbors(node)) - set(Au))
                for nb in nbs:
                    if random.random() <= p:
                        new_ones.append(nb)
            new_active = list(set(new_ones))
            Au += new_active
        spread.append(len(Au))
    return np.mean(spread)


#     influnces = seeds[:]
#     queue = influnces[:]
#     pre_node_record = defaultdict(float)
# #     print("Queue:",queue)
# #     print("Influences:",influnces)
#     while len(queue) != 0:
#         node = queue.pop(0)
# #         print("----------------------------------------------------------------------")
# #         print("Take node:",node)
# #         print("Neighbour:",graph[node])
#         for element in graph[node]:
#             if element not in influnces:
# #                 print("Element:",element,"prerecored",pre_node_record[element])
#                 pre_node_record[element] = pre_node_record[element] + graph[node][element]['weight']
# #                 print(pre_node_record[element])
#                 if pre_node_record[element] >= graph.nodes[element]['thres']:
# #                     print(">>>>>>>>>>>>>>>>>>node influeced:",element)
#                     influnces.append(element)
#                     queue.append(element)
# #     influnce_num = len(influnces)
# #     print("Seed set:",seeds,"Activated nodes:",influnces)
#     return len(influnces)
# # linear_Threshold(GG,[45,29])

#EDV
def Eval(G, S):
    Neighbors = Neighbor_Nodeset(G, S)

    fitness = len(S)
    L = list(set(Neighbors) - set(S))
    for TIME in L:
        fitness += 1 - (1 - 0.01) ** len(set(G.neighbors(TIME)) & set(S))
    return fitness

def Neighbor_nodes(G,u):
    return list(G.neighbors(u)) + [u]

def Neighbor_Nodeset(G,S):
    neighbors = [ ]
    for i in S:
        neighbors += Neighbor_nodes(G,i)
    neighbors = list(set(neighbors))
    return neighbors

def pop_init(nodes,pop,K,t,avg_d):
    P = []

    for i in range(pop):
        P_Item = []
        for Kt in range(K):
            temp = math.ceil((Kt+1) * math.exp(t * avg_d))+Kt+1
            if temp>len(nodes):
                temp = len(nodes)

            P_Item.append(nodes[random.randint(0,temp-1)])
        P.append(P_Item)

    return P

def mutation(G,P,P_remain,mu,K,t,avg_d,nodes):
    P_new = copy.deepcopy(P)

    for P_It in P_new:
        if P_new.index(P_It) == 0:
            temp = math.ceil(K * math.exp(t * avg_d))+K
            if temp <= K:
                temp += 1
            if temp>len(nodes):
                temp = len(nodes)

            ran_int = random.randint(0,K-1)

            while True:
                ran_temp = random.randint(0,temp-1)
                if nodes[ran_temp] not in P_It:
                    P_It[ran_int] = nodes[ran_temp]
                    break
            continue

        if P_new.index(P_It) < len(P_new)/2:
            temp = math.ceil(K * math.exp(t * avg_d))+K
        else:
            temp = math.ceil((K-1) * math.exp(t * avg_d))+K-1

        if temp <= K:
            temp += 1
        if temp > len(nodes):
            temp = len(nodes)

        # candidate nodes
        node_set = []
        # probability of candidate nodes being selected
        degree_pro = []

        for i in range(temp):
            node_set.append(nodes[i])
            degree_pro.append(G.degree(nodes[i]))

        degree_sum = sum(degree_pro)
        for deg in range(len(degree_pro)):
            degree_pro[deg] = degree_pro[deg] / degree_sum
        degree_pro = np.array(degree_pro)

        for index in range(len(P_It)):
            if P_It[index] not in P_remain[P_new.index(P_It)]:
                if random.random() < mu:
                    while True:
                        temp_d = np.random.choice(node_set, size=1, p=degree_pro.ravel())

                        if temp_d[0] not in P_It:
                            P_It[index] = temp_d[0]
                            break
    return P_new

def crossover(P,cr,pop):
    P_c = []
    P_remain = []

    random_int = 0
    P_c.append(P[random_int])

    P_remain.append([])
    for pop_index in range(pop):
        if pop_index == random_int:
            continue

        ind = list(set(P[random_int]) & set(P[pop_index]))
        P_remain.append(ind)
        P_1 = copy.deepcopy(P)
        P_1[random_int] = list(set(P_1[random_int]) - set(ind))
        P_1[pop_index] = list(set(P_1[pop_index]) - set(ind))

        for kk in range(len(P_1[random_int])):
            if random.random() < cr:
                ind.append(P_1[random_int][kk])
            else:
                ind.append(P_1[pop_index][kk])
        P_c.append(ind)
    return P_c,P_remain


def main(G,k):
#     for address in ["dolphins_edge_list.txt"]:
#         print(address)
    for K in [k]:
        result1 = []#influence spread
        result2 = []#running time
        for A in range(2):
#             print(strftime("%Y-%m-%d %H:%M:%S", localtime()))
#                 G = nx.read_edgelist(address, create_using=nx.Graph())
            start_time = time.perf_counter()

            nodes = list(G.nodes)
            edges = list(G.edges)
            nodes = sorted(nodes, key=lambda x: G.degree(x), reverse=True)

            pop = 10
            t = 0.04
            avg_d = round(2*len(edges)/len(nodes),2)
            mu = 0.1
            cr = 0.6
            maxgen = 15

            #initialization
            P = pop_init(nodes,pop,1,t,avg_d)

            for KK in range(2,K+1):
                i = 0
                #seed addition
                temp = math.ceil(KK * math.exp(t * avg_d)) + KK
                if temp>len(nodes):
                    temp = len(nodes)

                for ind in range(len(P)):
                    while True:
                        ran_index = random.randint(0,temp-1)
                        if nodes[ran_index] not in P[ind]:
                            P[ind].append(nodes[ran_index])
                            break

                while i < maxgen:
                    P = sorted(P,key=lambda x:Eval(G,x),reverse=True)

                    # mutation&crossover
                    P_cross, P_remain = crossover(P, cr, pop)
                    P_mutation = mutation(G,P_cross, P_remain, mu, KK, t, avg_d, nodes)

                    #selection
                    for index in range(pop):
                        Inf1 = Eval(G, P_mutation[index])
                        Inf2 = Eval(G, P[index])
                        if Inf1 > Inf2:
                            P[index] = P_mutation[index]
                    i += 1

            solution = sorted(P,key=lambda x:Eval(G,x),reverse=True)[0]

            end_time = time.perf_counter()
            runningtime = end_time-start_time

            result1.append(IC_model(G, solution, 10000))
            result2.append(runningtime)

#         print(result1)
#         print(result2)
#         print( "K:", K, "Influence spread:", round(np.mean(result1), 1),"Running time:",round(np.mean(result2),1))
#         print("seed set:",solution)
    return solution


# if __name__ == '__main__':
#     main()

def runSSR(G,k):
    seedset=main(G,k)
#     print("end:",seedset)
    return seedset




In [8]:
import networkx as nx
import random
import operator
from collections import deque

class SocialNetwork:
    def __init__(self, networkx_graph, num_seeds):
        self.networkx_graph = networkx_graph
        self.num_seeds = num_seeds

    def influence(self, seeds):
        influences = set(seeds)
        queue = deque(seeds)
        pre_node_record = defaultdict(float)
        while queue:
            node = queue.popleft()
            for neighbor in self.networkx_graph[node]:
                if neighbor not in influences:
                    pre_node_record[neighbor] = pre_node_record[neighbor] + self.networkx_graph[node][neighbor]['weight']
                    # Check if the neighbor is not already influenced and has reached the threshold
                    if pre_node_record[neighbor] >= self.networkx_graph.nodes[neighbor]['thres']:
                        influences.add(neighbor)
                        queue.append(neighbor)

        return len(influences)


class CuckooSearch:
    def __init__(self, social_network, comm_set_final, prev_seed_set, G_new, population_size=1000, levy_flights_exponent=1.5):
        self.social_network = social_network
        self.population_size = population_size
        # self.num_of_cuckoos = int(0.1 * population_size)
        self.num_of_cuckoos = random.randint(int(population_size/2),population_size)
        
        self.levy_flights_exponent = levy_flights_exponent
        self.res = {}

        closeness_centrality = nx.closeness_centrality(social_network.networkx_graph)
        degree_centrality = nx.degree_centrality(social_network.networkx_graph)

        candidate_nodes = set(G_new.nodes)
        new_candidate = list(candidate_nodes.union(prev_seed_set))
        self.new_candidate = new_candidate

        for node in new_candidate:
            x =self.social_network.influence([node])
            degree = degree_centrality[node]
            close = closeness_centrality[node]
            social_network.networkx_graph.nodes[node].update({'degree': degree, 'close': close, 'DV': x})
            social_network.networkx_graph.nodes[node]['res1'] = degree * x
            social_network.networkx_graph.nodes[node]['res2'] = close * x

        for node in new_candidate:
            self.res[node] = max(social_network.networkx_graph.nodes[node]['res1'], social_network.networkx_graph.nodes[node]['res2'])

            
#         print("Hello:",self.social_network)
        k = social_network.num_seeds
        self.population = [random.sample(new_candidate, k) for _ in range(population_size)]
        self.influence = {tuple(new_list): social_network.influence(new_list) for new_list in self.population}

        self.cuckoos = [random.sample(new_candidate, k) for _ in range(self.num_of_cuckoos)]
        self.cuckoo_influence = {tuple(new_list): social_network.influence(new_list) for new_list in self.cuckoos}

        best_cuckoo = max(self.cuckoo_influence.items(), key=operator.itemgetter(1))
        self.best_cuckoo = {best_cuckoo[0]: best_cuckoo[1]}

    def find_updated_solution(self, key, best_cuckoo):
        best_cuckoo = list(best_cuckoo)[0]
        best_cuckoo_nodes = set(best_cuckoo)
        new_candidate = list(set(key) | best_cuckoo_nodes)
        solu = {element: self.res[element] for element in new_candidate}

        sorted_items = sorted(solu.items(), key=lambda x: x[1], reverse=True)
        top_k_items = sorted_items[:len(key)]
        top_k_keys = [item[0] for item in top_k_items]
        return top_k_keys

    def update_population(self):
        cuckoo_key = list(self.best_cuckoo.keys())[0]
        best_cuckoo = set(cuckoo_key)
        best_cuckoo_inf = self.best_cuckoo[cuckoo_key]

        add_dict = {}
        dell_dict = {}

        for _ in range(self.population_size):
            key = random.choice(list(self.influence.keys()))
            if self.influence[key] < best_cuckoo_inf:
                new_solution = self.find_updated_solution(key, self.best_cuckoo)
                new_inf = self.social_network.influence(new_solution)

                if new_inf > self.influence[key]:
                    dell_dict.clear()
                    add_dict.clear()
                    dell_dict[key] = self.influence[key]
                    add_dict[tuple(new_solution)] = new_inf

                    new_key_value_pair = {tuple(new_solution): new_inf}
                    self.influence.update({tuple(new_solution): self.influence.pop(key)})
                    self.influence[tuple(new_solution)] = new_inf

                if new_inf > best_cuckoo_inf:
                    best_cuckoo_inf = new_inf
                    best_cuckoo = set(new_solution)
                    self.best_cuckoo.clear()
                    self.best_cuckoo[tuple(new_solution)] = best_cuckoo_inf

            else:
                best_cuckoo_inf = self.influence[key]
                best_cuckoo = set(key)
                self.best_cuckoo.clear()
                self.best_cuckoo[tuple(best_cuckoo)] = best_cuckoo_inf

            # Introduce randomness for diversity
            alien_finder = random.uniform(0, 1)
            if alien_finder < 0.25:
                if key in self.influence.keys():
                    new_list = random.sample(self.new_candidate, self.social_network.num_seeds)
                    new_inf = self.social_network.influence(new_list)
                    self.influence[tuple(new_list)] = self.influence.pop(key)
                    self.influence[tuple(new_list)] = new_inf

    def run(self, max_iterations=5):
#         for i in self.social_network.nodes():
#             print()
#         print("Hello:",self.social_network)
        for _ in range(max_iterations):
            self.update_population()
        return list(self.best_cuckoo.keys())[0]

def CSOrun(G, k, comm_set_final, prev_seed_set, G_new):
    print(G)
    social_network = SocialNetwork(G, k)
    cuckoo_search = CuckooSearch(social_network, comm_set_final, prev_seed_set, G_new)
    seed_set = cuckoo_search.run()
    return list(seed_set)



In [9]:

# Sort node names based on your desired range
def preprocess(G):
    current_names = list(sorted(G.nodes()))

    # Sort node names based on your desired range
    num_nodes = len(current_names)
    sorted_names = list(range(0, num_nodes))

    # Create a mapping from current names to sorted names
    mapping = {current_names[i]: sorted_names[i] for i in range(num_nodes)}

    # Rename nodes
    nx.relabel_nodes(G, mapping, copy=False)
    return G


# Now your graph nodes are renamed from 1 to the number of nodes
# You can print the nodes to verify
# print(sorted(G.nodes()))
# weighted_degrees = dict(nx.degree(G))


# In[6]:


def GetCommunities(file_path):
    b = {}
    list_of_lists = []
    file_path=comm_file_path
    # Open the file in read mode
    with open(file_path, 'r') as file:
        # Iterate through each line in the file
        # a = 0
        for line in file:
            # Strip the newline character from the end of the line and split by spaces
            elements = line.strip().split()
            b[int(elements[0])] = int(elements[1])
            # for i in range(len(elements)):
            #     b[int(elements[i])] = a
            # a+=1
    return b


def sort_nodes(nodes, centrality):
    """
    Sorts the Nodes in the given list of nodes based on the corresponding Centrality Measure of each node
    Parameters:
        nodes = List of nodes which has to be sorted
        centrality = List of Centrality Measures corresponding to each node in the list
    Returns:
        List of nodes sorted on the basis of their Centrality Measure in descending order
    """
    result = sorted(list(zip(centrality, nodes)), reverse = True)
    result = zip(*result)
    result = [list(tuple) for tuple in result]
    return result[1]

def sort_degree_centrality(G,nodes):
    """
    Calculates the Degree Centrality of each Node in the given list of nodes and returns a descendingly sorted list of nodes
    Parameters:
        nodes = List of nodes to be sorted on the basis of the Centrality Measure
    Returns:
        List of Nodes sorted in descending order on the basis of the Centrality Measure
    """
    centrality = nx.degree_centrality(G)
    return sort_nodes(nodes,centrality)

def sort_eigenvector_centrality(G,nodes):
    """
    Calculates the Eigenvector Centrality of each Node in the given list of nodes and returns a descendingly sorted list of nodes
    Parameters:
        nodes = List of nodes to be sorted on the basis of the Centrality Measure
    Returns:
        List of Nodes sorted in descending order on the basis of the Centrality Measure
    """
    centrality = nx.degree_centrality(G)
    cen_measure = []
    for i in nodes:
        cen_measure.append(centrality[i])
    return sort_nodes(nodes, cen_measure)

def sort_betweenness_centrality(G,nodes):
    """
    Calculates the Betweenness Centrality of each Node in the given list of nodes and returns a descendingly sorted list of nodes
    Parameters:
        nodes = List of nodes to be sorted on the basis of the Centrality Measure
    Returns:
        List of Nodes sorted in descending order on the basis of the Centrality Measure
    """
    centrality = nx.betweenness_centrality(G)
    return centrality

def sort_closeness_centrality(G,nodes):
    """
    Calculates the Closeness Centrality of each Node in the given list of nodes and returns a descendingly sorted list of nodes
    Parameters:
        nodes = List of nodes to be sorted on the basis of the Centrality Measure
    Returns:
        List of Nodes sorted in descending order on the basis of the Centrality Measure
    """
    centrality = nx.closeness_centrality(G)
    return centrality

def sort_katz_centrality(G,nodes):
    """
    Calculates the Katz Centrality of each Node in the given list of nodes and returns a descendingly sorted list of nodes
    Parameters:
        nodes = List of nodes to be sorted on the basis of the Centrality Measure
    Returns:
        List of Nodes sorted in descending order on the basis of the Centrality Measure
    """
    centrality = nx.degree_centrality(G)
    cen_measure = []
    for i in nodes:
        cen_measure.append(centrality[i])
    return sort_nodes(nodes, cen_measure)

def sort_percolation_centrality(G,nodes):
    """
    Calculates the Percolation Centrality of each Node in the given list of nodes and returns a descendingly sorted list of nodes
    Parameters:
        nodes = List of nodes to be sorted on the basis of the Centrality Measure
    Returns:
        List of Nodes sorted in descending order on the basis of the Centrality Measure
    """
    centrality = nx.degree_centrality(G)
    cen_measure = []
    for i in nodes:
        cen_measure.append(centrality[i])
    return sort_nodes(nodes, cen_measure)


# In[7]:


def one_hop_area(G,meme):

    temp = [] # Temporary list to store the One-Hop Area nodes of the meme
    for i in meme:
        temp.extend(list(G.neighbors(i)))
    return list(set(temp) - set(meme)) # Returning only unique nodes, to avoid Node Repetition

def two_hop_area(G,meme):

    one_hop = one_hop_area(G,meme)
    temp = [] # Temporary list to store the Two-Hop Area nodes of the meme
    for i in one_hop:
        temp.extend(list(G.neighbors(i)))
    return list(set(temp) - set(meme) - set(one_hop)) # Returning only unique nodes, to avoid Node Repetition

def calc_pcm_prob(G,nodes,weighted_degrees):
    prob = [] # Temporary List to store the corresponding Cascade Probability of the Nodes
    for i in nodes:
        prob.append(weighted_degrees[i] / G.number_of_nodes())
    return prob

def calc_edges(G,group1, group2):
    count = [] # Temporary storage to store the Number of edges corresponding each node
    for i in group2:
        edges = list(nx.edges(G, nbunch = [i]))
        temp = 0
        for i in edges:
            if (i[1] in group1) or (i[1] in group2):
                temp += 1
        count.append(temp)
    return count

def sum_pd(list1, list2):
    p = 0 # Temporary variable to store the Sum-Product of the two lists
    for i in range(len(list1)):
        p += (list1[i] * list2[i])
    return p

def common_neighbors_directed(graph, u, v):
    # Ensure the graph is directed
    if not graph.is_directed():
        raise ValueError("The graph must be directed.")
    
    # Predecessors of u and v
    pred_u = set(graph.predecessors(u))
    pred_v = set(graph.predecessors(v))
    
    # Successors of u and v
    succ_u = set(graph.successors(u))
    succ_v = set(graph.successors(v))
    
    # Common neighbors are those nodes that are both successors and predecessors of u and v
    common_neighbors = (pred_u & pred_v) | (succ_u & succ_v)
    
    return common_neighbors

def calc_edge_prob(G,meme, one_hop):
    N = G.number_of_nodes() # Total Nodes in the Graph
    prob_sum = 0
    for i in one_hop:
        prob_prod = 1
        for j in meme:
            pij = 0.01
            pij += ((G.degree(i) + G.degree(j)) / N)
            pij += (len(list(common_neighbors_directed(G, i, j))) / N)
            prob_prod *= (1 - pij)
        prob_sum += (1 - prob_prod)
    return prob_sum

def LIE(G,meme,weighted_degrees):
    k=len(meme)
    Ns1_S = one_hop_area(G,meme) # One-Hop area of the Meme
    Ns2_S = two_hop_area(G,meme) # Two-Hop Area of the Meme
    pu = calc_pcm_prob(G,Ns2_S,weighted_degrees)
    du = calc_edges(G,Ns1_S, Ns2_S)
    return  k + ((1 + ((1 / len(Ns1_S)) * sum_pd(pu, du))) * calc_edge_prob(G,meme, Ns1_S)) if len(Ns1_S) !=0 else 0


# In[8]:


def pareto_dominance(seed_set_x, seed_set_y):
    """Check if seed set x dominates seed set y."""
    all_less_equal = seed_set_x['F'] >=seed_set_y['F'] and seed_set_x['sigma'] >= seed_set_y['sigma']
    any_strictly_greater = seed_set_x['F'] > seed_set_y['F'] or seed_set_x['sigma'] > seed_set_y['sigma']
    return all_less_equal and any_strictly_greater

def pareto_optimal_seed_sets(seed_sets):
    """Identify Pareto-optimal seed sets from a list of seed sets."""
    pareto_optimal_sets = []
    for seed_set_x in seed_sets:
        if not any(pareto_dominance(seed_set_y, seed_set_x) for seed_set_y in seed_sets if seed_set_x != seed_set_y):
            pareto_optimal_sets.append(seed_set_x)
    return pareto_optimal_sets

def pareto_rank(seed_sets):
    """Assign Pareto rank to seed sets."""
    ranks = [0] * len(seed_sets)
    current_rank = 0
    while True:
        pareto_sets = []
        for i, seed_set in enumerate(seed_sets):
            if ranks[i] != 0:
                continue  # Already assigned a rank
            if all(not pareto_dominance(seed_set, other_set) for j, other_set in enumerate(seed_sets) if i != j and ranks[j] == 0):
                pareto_sets.append(seed_set)
                ranks[i] = current_rank + 1
        if not pareto_sets:
            break
        current_rank += 1
    for i in range(len(seed_sets)):
      seed_sets[i]['rank'] = ranks[i]


def crowding_distance(seed_sets):
    """Calculate crowding distance for seed sets based on their objectives."""
    for seed_set in seed_sets:
        seed_set['distance'] = 0
    seed_sets.sort(key=lambda x: x['F'])
    seed_sets[0]['distance'] = float('inf')
    seed_sets[len(seed_set)-1]['distance'] = float('inf')
    rnge = seed_sets[len(seed_sets)-1]['F'] - seed_sets[0]['F']
    for i in range(1,len(seed_sets)-1):
      if rnge != 0:
        seed_sets[i]['distance'] += (seed_sets[i+1]['F'] - seed_sets[i-1]['F'])/ rnge
      else:
        seed_sets[i]['distance'] = float('inf')

    seed_sets.sort(key = lambda x: x['sigma'])
    seed_sets[0]['distance'] = float('inf')
    seed_sets[len(seed_sets)-1]['distance'] = float('inf')
    rnge = seed_sets[len(seed_sets)-1]['sigma'] - seed_sets[0]['sigma']
    for i in range(1,len(seed_sets)-1):
      if rnge != 0:
        seed_sets[i]['distance'] += (seed_sets[i+1]['sigma'] - seed_sets[i-1]['sigma'])/ rnge
      else:
        seed_sets[i]['distance'] = float('inf')


def select_seed_sets(candidate_sets, Q):
    """Select Q seed sets from candidate sets."""
    selected = []
#     print(candidate_sets)
    candidate_sets.sort(key=lambda x: (-x['rank'], x['distance']))
    selected = candidate_sets[:Q]
    return selected



# In[9]:


import numpy as np
import time


# In[ ]:





# In[10]:


import collections
def divide_nodes_into_groups(n, m):
    if n.number_of_nodes() < m:
        raise ValueError("Number of nodes must be greater than or equal to the number of groups.")

    # Initialize an empty list of groups
    groups = [[] for _ in range(m)]

    # Assign each node to a random group
    nodes = list(n.nodes())
    random.shuffle(nodes)
    for i, node in enumerate(nodes):
        group_index = i % m  # Cycle through groups
        groups[group_index].append(node)

    return groups

def simulate_diffusion(graph, seeds, p=0.8):
    influenced_nodes = set(seeds)
    new_nodes = set(seeds)

    while new_nodes:
        current_nodes = new_nodes.copy()
        new_nodes = set()

        for node in current_nodes:
            neighbors = set(graph.neighbors(node)) - influenced_nodes
            for neighbor in neighbors:
                edge_weight = graph[node][neighbor]['weight']
                if edge_weight >= p:
                    new_nodes.add(neighbor)
                    influenced_nodes.add(neighbor)
    return list(influenced_nodes)

def calculate_hoover_index(network, seed_set,groups):
    influenced_nodes = simulate_diffusion(network, seed_set)
    # Calculate fractions of influenced nodes in each group
    fractions = [len(set(group) & set(influenced_nodes)) / len(group) for group in groups]
    N = len(fractions)
    T = 0
    A = 0
    for i in range (0,N):
        T += fractions[i]
        for j in range (0,len(fractions)):
            A = A + abs(fractions[i] - fractions[j])

    hoover_index = (A)/(2*N*T)
    # print(hoover_index)
    return hoover_index

def calculate_influence_spread(network, seed_set):
    d = simulate_diffusion(network,seed_set)
    # print(d)
    return len(d)
    

def calculate_F(network, seed_set,groups):
    """Calculate fairness metric F(S) for a given seed set."""
    # gini_coefficient = calculate_gini_coefficient(network, seed_set,0.5)
    hoover_index = calculate_hoover_index(network,seed_set,groups)

    return 1 - hoover_index

def calculate_sigma(network, seed_set):
    """Calculate influence spread sigma(S) for a given seed set."""
    weighted_degrees = dict(nx.degree(network))
    return LIE(network,seed_set,weighted_degrees)

def seed_set_evaluation(network, seed_sets,groups,b):
    """Evaluate F(S) and sigma(S) for a list of seed sets."""
    info = []
    
    for seed_set in seed_sets:
        F = calculate_F(network, seed_set,groups)
        sigma = calculate_sigma(network, seed_set)
        inf_spr = calculate_influence_spread(network,seed_set)
        
        info.append({'seed':seed_set,'F':F,'sigma':sigma, 'spread':inf_spr})
#     qwe = set()
#     for i in range(len(inf)):
#         qwe.add(b.get(int(inf[i]),0))
    # print(qwe)
    return info


# In[11]:
def uniform_mutation(seed_set, mutation_probability, nodes):
    """Apply uniform mutation to a seed set."""
    mutated_seed_set = []
    for gene in seed_set:
        if random.random() < mutation_probability:
            new_gene = random.choice(list(set(nodes) - set(seed_set)))
            mutated_seed_set.append(new_gene)
        else:
            mutated_seed_set.append(gene)
    return mutated_seed_set

def uniform_crossover(parent1, parent2, crossover_probability):
    """Apply uniform crossover between two parent seed sets."""
    if random.random() < crossover_probability:
        child1, child2 = [], []
        for gene1, gene2 in zip(parent1, parent2):
            if random.random() < 0.5:
                child1.append(gene1)
                child2.append(gene2)
            else:
                child1.append(gene2)
                child2.append(gene1)
        return child1, child2
    else:
        return parent1, parent2

def initialize_particle(G,npop,k):
    position=[]
    for i in range(0,npop):
      position.append(random.sample(range(G.number_of_nodes()),k))
    return position

def initialize_populationf(num_individuals, nodes,k):
    """Initialize the population with a combination of random and prior seed sets."""
    population = []
    for _ in range(num_individuals):
        seed_set = random.sample(list(nodes), k)
        population.append(seed_set)
    return population

# In[12]:
def influence_maximization_with_fairness(num_generations,network, num_individuals,  M_p, C_p, Q,groups,b):
    """Run influence maximization considering fairness."""
    # Preprocessing
    prior_knowledge = list(sort_degree_centrality(network,network.nodes()))
    prior_knowledge = prior_knowledge[:Q]
    # Initialization
    population = initialize_populationf(num_individuals, network.nodes(), Q)
#     print(population)
    population.append(prior_knowledge)
#     print(population)
    info = []
    for _ in range(1, num_generations+1):
        # Seed Set Evolution
        offspring_population = []
        for _ in range(num_individuals):
            parent1, parent2 = random.sample(population, 2)
            child1, child2 = uniform_crossover(parent1, parent2, C_p)
            child1 = uniform_mutation(child1, M_p, network.nodes())
            child2 = uniform_mutation(child2, M_p, network.nodes())
            offspring_population.extend([child1, child2])

        # Seed Set Evaluation
        seed_sets = offspring_population
#         print(seed_sets)
        info = seed_set_evaluation(network, seed_sets,groups,b)
        pareto_rank(info)
        crowding_distance(info)
    # Selection
    population = select_seed_sets(info, 1)
    return population

def jaya_optimize(G,k,npop, max_iter, num_nodes,groups,b):
    swarm = initialize_particle(G,npop,k)
    # print(swarm)
    # return
    prev_swarm = copy.deepcopy(swarm)
    info = []
    ert = set()
    # print(max_iter)
    for iter in range(max_iter):
        # print(swarm)
        best_fitness = swarm[0]
        worst_fitness = swarm[0]
        # print(type(best_fitness))
        # return
        b_val = calculate_influence_spread(G,best_fitness)
        w_val = calculate_influence_spread(G,worst_fitness)
        for i in range(len(swarm)):
          val = calculate_influence_spread(G,swarm[i])
          if val > b_val:
             best_fitness = swarm[i]
             b_val = val
          elif val < w_val:
             worst_fitness = swarm[i]
             w_val = val
        for i, particle in enumerate(swarm):
          fitness = calculate_influence_spread(G,particle)
          current = []
          present = set()
          not_present = set(G.nodes() - particle)
          for j in range(k):
              update_val = particle[j] + (random.random())*(best_fitness[j] - particle[j]) - (random.random())*(worst_fitness[j] - particle[j])
              update_val = round(update_val)
              if update_val in present:
                  update_val = random.choice(list(not_present))
                  present.add(update_val)
                  not_present.remove(update_val)
              elif update_val not in not_present:
                  update_val = random.choice(list(not_present))
                  present.add(update_val)
                  not_present.remove(update_val)
              current.append(update_val)
          new_fitness = calculate_influence_spread(G,current)
          if new_fitness > fitness:
              swarm[i] = current
        seed_sets = swarm + prev_swarm
        info = seed_set_evaluation(G, seed_sets,groups,b)
        # print(info)
        pareto_rank(info)
        crowding_distance(info)
    # Selection
    # print(f"fjsdbjsfkbjg :  {info}")
    population = select_seed_sets(info, 1)
    return population,ert


# In[22]:
def m_optimize(G,k,npop, max_iter, num_nodes,groups,b):
    swarm = initialize_particle(G,npop,k)
    prev_swarm = copy.deepcopy(swarm)
    memory = initialize_particle(G,npop,k)
    info = []
    # print(max_iter)
    ert = set()
    flmx = G.number_of_nodes()
    flmn = 0
    for iter in range(max_iter):
        DAP = 1 - iter/max_iter
        Dfl = flmx - (flmx - flmn)* DAP
        for i in range(len(swarm)):
            present = set()
            not_present = set(G.nodes() - swarm[i])
            for j in range(len(swarm)):
                if i==j:
                    continue
                rand_num = random.random()
                for c in range(k):
                    if DAP <= rand_num:
                        update_val = round(swarm[i][c] + random.random() * Dfl * (memory[j][c] - swarm[i][c]))
                        if update_val not in not_present and len(not_present)!=0:
                            swarm[i][c] = random.choice(list(not_present))
                        else:
                            swarm[i][c] = update_val
                    else:
                        if(len(not_present)!=0):
                            swarm[i][c] = random.choice(list(not_present))
                    present.add(swarm[i][c])
                    not_present.remove(swarm[i][c])

        fitness = calculate_influence_spread(G,swarm[i])
        m_fitness = calculate_influence_spread(G,memory[i])
        if fitness > m_fitness:
            memory[i] = swarm[i]
        seed_sets = swarm + prev_swarm
        info = seed_set_evaluation(G, seed_sets,groups,b)
        # print(info)
        pareto_rank(info)
        crowding_distance(info)
    # Selection
    # print(f"fjsdbjsfkbjg :  {info}")
    population = select_seed_sets(info, 1)
    return population,ert

def runJAYA(G,k,Communities):
    G=preprocess(G)
    npop = 5
    max_iter = 5
    num_nodes = G.number_of_nodes()
#     k = 5
    groups = divide_nodes_into_groups(G, 4)
    # for e in G.edges():
    #   G.edges[e]['weight'] = round(random.uniform(0.01,1.0),2)
    # for n in G.nodes():
    #   G.nodes[n]['t'] = G.degree(n)/2
    # starttime = time.time()
    b=Communities
    # print(Communities)
    info,ans=jaya_optimize(G,k, npop, max_iter, num_nodes,groups,b)
#     print(info[0]['seed'])
    return info[0]['seed']
# k=10
# file_path = 'LFR_1000_0.2_comm_list.txt'  # Replace with the path to your file
# CommunitySet = GetCommunities(file_path)
# runJAYA(G,k,CommunitySet)

def runMOCSA(G,k,Communities):
    npop = 10
    max_iter = 5
    num_nodes = G.number_of_nodes()
    groups = divide_nodes_into_groups(G, 4)
    b=Communities
    info,ans=m_optimize(G,k, npop, max_iter, num_nodes,groups,b)
#     print(info[0]['seed'])
    return info[0]['seed']

def runFIMMOGA(G,k,Communities):
    network = G
    generations = 5
    num_individuals = 5
    M_p = 0.5
    C_p = 0.5
    Q = k
    groups = divide_nodes_into_groups(G,4)
    # p = np.zeros((G.number_of_nodes(),G.number_of_nodes()))
    # for e in G.edges():
    #   n = e[1]
    #   if G.degree[n] > 0:
    #     p[e[0],e[1]] =  round(random.uniform(0.01,1.0),2)
    #   else:
    #     p[e[0],e[1]] =  round(random.uniform(0.01,1.0),2)

    # psum = p.sum(axis=0)

    # for e in G.edges():
    # G.edges[e]['p'] = round(random.uniform(0.01,1.0),2)

    # for n in G.nodes():
    # G.nodes[n]['t'] = G.degree(n)/2
    # st = time.time()
    b = Communities
    info=influence_maximization_with_fairness(generations, network, num_individuals,  M_p, C_p, Q,groups,b)
    # ft = time.time()-st
    # print(info)
    # print(len(ans))
    # print(ft)
    # print(len(b))
#     print(info[0]['seed'])
    return info[0]['seed']


In [10]:
''' Implementation of PMIA algorithm [1].
[1] -- Scalable Influence Maximization for Prevalent Viral Marketing in Large-Scale Social Networks.
'''


def updateAP(ap, S, PMIIAv, PMIIA_MIPv, Ep):
    ''' Assumption: PMIIAv is a directed tree, which is a subgraph of general G.
    PMIIA_MIPv -- dictionary of MIP from nodes in PMIIA
    PMIIAv is rooted at v.
    '''
    # going from leaves to root
    sorted_MIPs = sorted(PMIIA_MIPv.items(), key = lambda MIP: len(MIP), reverse = True)
#     print("Edges:",PMIIAv.nodes)
#     for e in PMIIAv.edges:
#         print(e)
#     for u,_ in sorted_MIPs:
#         print(u,_)
        
        
    for u, _ in sorted_MIPs:
        if u in S:
            ap[(u, PMIIAv)] = 1
        elif not PMIIAv.in_edges([u]):
#             print("\n\nGoin in elif")
            ap[(u, PMIIAv)] = 0
        else:
            in_edges = PMIIAv.in_edges([u], data=True)
            prod = 1
            for w, _, edata in in_edges:
                # p = (1 - (1 - Ep[(w, u)])**edata["weight"])
                
                print("In updateAP")
                print("hello:",w,PMIIAv)
                if (w,PMIIAv) in ap.keys():
                    print("Key present")
                else:
                    print("Key is NOT present")
#                 print("keys:",
    
                p = Ep[(w,u)]
                prod *= 1 - ap[(w, PMIIAv)]*p
#             print(ap,u, PMIIAv)
            ap[(u, PMIIAv)] = 1 - prod

def updateAlpha(alpha, v, S, PMIIAv, PMIIA_MIPv, Ep, ap):
    # going from root to leaves
    sorted_MIPs =  sorted(PMIIA_MIPv.items(), key = lambda MIP: len(MIP))
    for u, mip in sorted_MIPs:
        if u == v:
            alpha[(PMIIAv, u)] = 1
        else:
            out_edges = PMIIAv.out_edges([u])
            assert len(out_edges) == 1, "node u=%s must have exactly one neighbor, got %s instead" %(u, len(out_edges))
            out_edges=list(out_edges)
#             print("out_edges:",out_edges,type(out_edges))
            
            w = out_edges[0][1]
            if w in S:
                alpha[(PMIIAv, u)] = 0
            else:
                in_edges = PMIIAv.in_edges([w], data=True)
                prod = 1
                for up, _, edata in in_edges:
                    if up != u:
                        # pp_upw = 1 - (1 - Ep[(up, w)])**edata["weight"]
                        pp_upw = Ep[(up, w)]
                        prod *= (1 - ap[up]*pp_upw)
                # alpha[(PMIIAv, u)] = alpha[(PMIIAv, w)]*(1 - (1 - Ep[(u,w)])**PMIIAv[u][w]["weight"])*prod
                alpha[(PMIIAv, u)] = alpha[(PMIIAv, w)]*(Ep[(u,w)])*prod

def computePMIOA(G, u, theta, S, Ep):
    '''
     Compute PMIOA -- subgraph of G that's rooted at u.
     Uses Dijkstra's algorithm until length of path doesn't exceed -log(theta)
     or no more nodes can be reached.
    '''
    # initialize PMIOA
    PMIOA = nx.DiGraph()
    PMIOA.add_node(u)
    PMIOA_MIP = {u: [u]} # MIP(u,v) for v in PMIOA

    crossing_edges = set([out_edge for out_edge in G.out_edges([u]) if out_edge[1] not in S + [u]])
    edge_weights = dict()
    dist = {u: 0} # shortest paths from the root u

    # grow PMIOA
    while crossing_edges:
        # Dijkstra's greedy criteria
        min_dist = float("Inf")
        sorted_crossing_edges = sorted(crossing_edges) # to break ties consistently
        for edge in sorted_crossing_edges:
            if edge not in edge_weights:
                # edge_weights[edge] = -math.log(1 - (1 - Ep[edge])**G[edge[0]][edge[1]]["weight"])
                edge_weights[edge] = -math.log(Ep[edge])
            edge_weight = edge_weights[edge]
            if dist[edge[0]] + edge_weight < min_dist:
                min_dist = dist[edge[0]] + edge_weight
                min_edge = edge
        # check stopping criteria
        if min_dist < -math.log(theta):
            dist[min_edge[1]] = min_dist
            # PMIOA.add_edge(min_edge[0], min_edge[1], {"weight": G[min_edge[0]][min_edge[1]]["weight"]})
            PMIOA.add_edge(min_edge[0], min_edge[1])
            PMIOA_MIP[min_edge[1]] = PMIOA_MIP[min_edge[0]] + [min_edge[1]]
            # update crossing edges
            crossing_edges.difference_update(G.in_edges(min_edge[1]))
            crossing_edges.update([out_edge for out_edge in G.out_edges(min_edge[1])
                                   if (out_edge[1] not in PMIOA) and (out_edge[1] not in S)])
        else:
            break
    return PMIOA, PMIOA_MIP

def updateIS(IS, S, u, PMIOA, PMIIA):
    for v in PMIOA[u]:
        for si in S:
            # if seed node is effective and it's blocked by u
            # then it becomes ineffective
            if (si in PMIIA[v]) and (si not in IS[v]) and (u in PMIIA[v][si]):
                    IS[v].append(si)

def computePMIIA(G, ISv, v, theta, S, Ep):

    # initialize PMIIA
    PMIIA = nx.DiGraph()
    PMIIA.add_node(v)
    PMIIA_MIP = {v: [v]} # MIP(u,v) for u in PMIIA

    crossing_edges = set([in_edge for in_edge in G.in_edges([v]) if in_edge[0] not in ISv + [v]])
    edge_weights = dict()
    dist = {v: 0} # shortest paths from the root u

    # grow PMIIA
    while crossing_edges:
        # Dijkstra's greedy criteria
        min_dist = float("Inf")
        sorted_crossing_edges = sorted(crossing_edges) # to break ties consistently
        for edge in sorted_crossing_edges:
            if edge not in edge_weights:
                # edge_weights[edge] = -math.log(1 - (1 - Ep[edge])**G[edge[0]][edge[1]]["weight"])
#                 edgevalue=Ep[edge]
#                 print("Ep[edge]:",Ep[edge],edgevalue)
                edge_weights[edge] = -(math.log(Ep[edge]))
#                 edge_weights[edge] = -(math.log(edgevalue))
                
            edge_weight = edge_weights[edge]
            if dist[edge[1]] + edge_weight < min_dist:
                min_dist = dist[edge[1]] + edge_weight
                min_edge = edge
        # check stopping criteria
        # print min_edge, ':', min_dist, '-->', -math.log(theta)
        if min_dist < -math.log(theta):
            dist[min_edge[0]] = min_dist
            # PMIIA.add_edge(min_edge[0], min_edge[1], {"weight": G[min_edge[0]][min_edge[1]]["weight"]})
            PMIIA.add_edge(min_edge[0], min_edge[1])
            PMIIA_MIP[min_edge[0]] = PMIIA_MIP[min_edge[1]] + [min_edge[0]]
            # update crossing edges
            crossing_edges.difference_update(G.out_edges(min_edge[0]))
            if min_edge[0] not in S:
                crossing_edges.update([in_edge for in_edge in G.in_edges(min_edge[0])
                                       if (in_edge[0] not in PMIIA) and (in_edge[0] not in ISv)])
        else:
            break
    return PMIIA, PMIIA_MIP

def PMIA(G, k, theta, Ep):
    start = time.time()
    # initialization
    S = []
    IncInf = dict(zip(G.nodes(), [0]*len(G)))
    PMIIA = dict() # node to tree
    PMIOA = dict()
    PMIIA_MIP = dict() # node to MIPs (dict)
    PMIOA_MIP = dict()
    ap = dict()
    alpha = dict()
    IS = dict()
    for v in G:
        IS[v] = []
        PMIIA[v], PMIIA_MIP[v] = computePMIIA(G, IS[v], v, theta, S, Ep)
        for u in PMIIA[v]:
            ap[u] = 0 # ap of u node in PMIIA[v]
        updateAlpha(alpha, v, S, PMIIA[v], PMIIA_MIP[v], Ep, ap)
        for u in PMIIA[v]:
            IncInf[u] += alpha[(PMIIA[v], u)]*(1 - ap[u])
#     print('Finished initialization')
#     print(time.time() - start)

    # main loop
    for i in range(k):
#         print(IncInf)
#         u, _ = max(IncInf.items(), key = lambda dk, dv: dv)
        _,u=max(zip(IncInf.values(), IncInf.keys()))
        IncInf.pop(u) # exclude node u for next iterations
        PMIOA[u], PMIOA_MIP[u] = computePMIOA(G, u, theta, S, Ep)
        for v in PMIOA[u]:
            for w in PMIIA[v]:
                if w not in S + [u]:
                    IncInf[w] -= alpha[(PMIIA[v],w)]*(1 - ap[w])

        updateIS(IS, S, u, PMIOA_MIP, PMIIA_MIP)

        S.append(u)

        for v in PMIOA[u]:
            if v != u:
                PMIIA[v], PMIIA_MIP[v] = computePMIIA(G, IS[v], v, theta, S, Ep)
                
                
                for uu in PMIIA[v].nodes:
                    if uu in S:
#                         print("IN s")
                        ap[u] = 1
                    elif not PMIIA[v].in_edges([uu]):
#                         print("Second else if")
            #             print("\n\nGoin in elif")
                        ap[uu] = 0
                    else:
                        in_edges = PMIIA[v].in_edges([uu], data=True)
                        prod = 1
#                         print("inEdgessss:",in_edges)
                        for w, _, edata in in_edges:
                            # p = (1 - (1 - Ep[(w, u)])**edata["weight"]
                            p = Ep[(w,uu)]
                            prod *= 1 - ap[w]*p
                        ap[uu] = 1 - prod
                
#                 updateAP(ap, S, PMIIA[v], PMIIA_MIP[v], Ep)
                updateAlpha(alpha, v, S, PMIIA[v], PMIIA_MIP[v], Ep, ap)
                # add new incremental influence
                for w in PMIIA[v]:
                    if w not in S:
                        IncInf[w] += alpha[(PMIIA[v], w)]*(1 - ap[w])

    return S

def getCoverage(G, S, Ep):
    return IC(G, S)


def run_PMIA(GG,k):
    Ep = dict()
    G=nx.DiGraph()
    for edgee in GG.edges():
        s=edgee[0]
        t=edgee[1]
        Ep[(int(s), int(t))] = GG[s][t]['weight']
        G.add_edge(s,t,weight=GG[s][t]['weight'])   
        G.nodes[s]['thres']=(G.degree(s)/2)
        G.nodes[t]['thres']=(G.degree(t)/2)    
    theta = 1.0/20
    S = PMIA(G, k, theta, Ep)
    return S


In [11]:
def Greedy(G,k):
    print("in Greedy")
    Dict={}
    mySet1=[]
    V=G.nodes()
    mySet1.clear()
    for i in range(k):
        for v in (V-mySet1):
            mySet1.append(v)
            a=linear_Threshold(G,mySet1)
            Dict[v]=len(a)#influence as value and current node as key
            mySet1.remove(v)#remove crrent node from mySet for rest nodes to go for IC
        Keymax = max(zip(Dict.values(), Dict.keys()))[1]# finding node with max influence
        Dict.clear()
        mySet1.append(Keymax)
#     print("Final seed set is:",mySet1,compute_Phi(G,mySet1,comm,k))
    return list(mySet1)


In [12]:
def read_txt(path):
  file1 = open(path,'r')
  sender = list()
  receiver = list()

  for i in file1.readlines():
    sender.append(int(i.split(' ')[0]))
    receiver.append(int(i.split(' ')[1].split('\n')[0]))
    
  df = pd.DataFrame(list(zip(sender,receiver)),columns =['source', 'target'])
  return df

In [13]:
def gmltotxt(filename):
    import networkx as nx
    import pandas as pd
    g = nx.read_gml('airlines.gml')
    nx.write_edgelist(g, 'edgelistFile.csv', delimiter=',')
    df = pd.read_csv('edgelistFile.csv')
    file = open("myfile.txt","w")
    for i in range(len(df)):
        x=df.iloc[i][0]
        y=df.iloc[i][1]
        file.write(str(x)+" "+str(y)+"\n")
    file.close()

In [14]:
def getcand(G,k, comm):#Take df and  all new nodes as input and return a seed node.
    s=[]
    s=GreedyDiv(G,k, comm)
    return s

In [15]:
def GreedyDiv(G,k, comm):
    Dict={}
    mySet1=[]
    V=G.nodes()
    mySet1.clear()
#     print("Community:",comm,"k:",k)
    for i in range(k):
#         print("myset:",mySet1)
        for v in (V-mySet1):
#             print("v,i:",v,i)
            mySet1.append(v)
#             print("myset:",mySet1)
            a=compute_Phi(G,mySet1,comm,i+1)
#             print("Phi of ",mySet1," is :",a)
            Dict[v]=a#influence as value and current node as key
            mySet1.remove(v)#remove crrent node from mySet for rest nodes to go for IC
               #print(Dict)
        Keymax = max(zip(Dict.values(), Dict.keys()))[1]# finding node with max influence
        Dict.clear()
        mySet1.append(Keymax)
#     print("Final seed set is:",mySet1,compute_Phi(G,mySet1,comm,k))
    return list(mySet1)


In [16]:
def linear_Threshold(graph, seeds):
    influnces = seeds[:]
    queue = influnces[:]
    pre_node_record = defaultdict(float) 
#     print("Queue:",queue)
#     print("Influences:",influnces)
    while len(queue) != 0:
        node = queue.pop(0)
#         print("----------------------------------------------------------------------")
#         print("Take node:",node)
#         print("Neighbour:",graph[node])
        for element in graph[node]:
            if element not in influnces:
#                 print("Element:",element,"prerecored",pre_node_record[element])
                pre_node_record[element] = pre_node_record[element] + graph[node][element]['weight'] 
#                 print(pre_node_record[element])
                if pre_node_record[element] >= graph.nodes[element]['thres']:
#                     print(">>>>>>>>>>>>>>>>>>node influeced:",element)
                    influnces.append(element)
                    queue.append(element)
#     influnce_num = len(influnces)
#     print("Seed set:",seeds,"Activated nodes:",influnces)
    return influnces
# linear_Threshold(GG,[45,29])

In [17]:
# def communityDiversityFunction(G, S, communities):
#     activated = linear_Threshold(G, S)
#     noofcommunity=0
#     outersum=0
#     for com in communities:
#         innersum=0
#         for rvj in com:
#             if rvj in activated:
#                 innersum=innersum+1
#         outersum=outersum+math.sqrt(innersum)
#     return outersum

def communityDiversityFunction(G, S, communities):
    activated = linear_Threshold(G, S)
    noofcommunity=0
    for com in communities:
        if any(x in activated for x in com):
            noofcommunity=noofcommunity+1
    return noofcommunity

In [18]:
def compute_Phi(G, S, communities, k):
    if len(S)<1:
        return 0
    lambda_G = 0.5
    
    v_length = G.number_of_nodes()
    diversity_V = communityDiversityFunction(G, list(G.nodes), communities)

    IC_S = linear_Threshold(G, S)
    activated_set_S_length = len(IC_S) 
    diversity_activated_set_S = communityDiversityFunction(G, IC_S, communities)
    phi_S = ((1 - lambda_G)* (activated_set_S_length/v_length)) + (lambda_G * (diversity_activated_set_S/diversity_V))
    return phi_S
    
    
    


In [19]:

# obj=TBCD_mine()
# perc=[20,40,60,80]
# filename='200_0'
# print("hello")
# obj.execute_TBCD_txt(filename,perc)

In [20]:
def findk(i):#dynamic calculation of k according the percentage of current dataset
  k=(0.003*i)
  if(i==0):
    k=1
  if(k>int(k)):
   k=int(k)+1
  return k

In [21]:
def displayresult(result,itr,perc):
    print("\n\n\n\n^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^")

    linestylee=['dashdot','dashed','dotted','-', '--', ':','-.','dashdot','dashed','dotted','-', '--', ':','-.' ]
    
    markerss = ['o','v','s','*','+','x','D','d','X','P','x','D','d','X','P']
#     descriptions = ['circle', 'triangle_down','square','star', 'plus','x','diamond', 'thin_diamond','x (filled)','plus (filled)']
    communitiesITR=[]
    Name=[]
    activatednodesITR=[]
    totalcomm=[]
#     print(itr,result)
    
    for i in result:
        for j in range(len(result[i])):
            Name.append(result[i][j]['Name'])
        break
#     print(Name)
    
#     for i in result:
# #         print(result[i])
#         for j in range(len(result[i])):
#             totalcomm.append(result[i][j]['Total communitites'])
#             break
#     print("Total communitites:",totalcomm)
    
    for i in result:
        active=[]
        comm=[]
        for j in range(len(result[i])):
            active.append(result[i][j]['length of activated nodes'])
            comm.append(result[i][j]['number of communities'])
        activatednodesITR.append(active)
        communitiesITR.append(comm)
#     print("Activated nodes:",activatednodesITR)
#     print("Communities:",communitiesITR)
        
    
        
#     print("hello");
    
    plt.xlabel("Percentage")
    plt.ylabel("Activated nodes")
    plt.title("Activated nodes - LFR_1000_0.0 - Greedy - lemda=0.5 - Degree")
    for i in range(len(activatednodesITR[0])):
        plt.plot(perc,[pt[i] for pt in activatednodesITR],label = '%s'%Name[i],linestyle='%s'%linestylee[i],marker='%s'%markerss[i])
    plt.legend()
    plt.show()
    
    
    plt.xlabel("Percentage")
    plt.ylabel("No of community")
    plt.title("Communitites - LFR_1000_0.0 - Greedy - lemda=0.5 - Degree")
    for i in range(len(communitiesITR[0])):
        plt.plot(perc,[pt[i] for pt in communitiesITR],label = '%s'%Name[i],linestyle='%s'%linestylee[i],marker='%s'%markerss[i])
#     plt.plot(perc,totalcomm,label='Total community',linestyle='%s'%linestylee[-1],marker='%s'%markerss[-1])
    plt.legend()
    plt.show()
    

In [22]:
def findcommunity(G,seedset,k, comm,algoname):
    print ("Communities formed: ",len(comm))
    activated = linear_Threshold(G, seedset)
    noofcommunity=0
    for com in comm:
        if any(x in activated for x in com):
            noofcommunity=noofcommunity+1
    print("Community we got:",noofcommunity)
    upperBound_dict = {
        'Name':algoname,
        'k_nodes': k,
#         'Total communitites':len(comm),
        'number of communities':noofcommunity,
        'length of activated nodes': len(activated),
        
        # 'length of communities': len(community_df['Unnamed: 1'].unique())
    }
    print(upperBound_dict)
    return upperBound_dict

In [23]:
def findresult(G,k,comm_set_final):
    #TBCD
    print('\n\n--------------- DCDIM -------------------')
    R_seed = [getcand(G,int(k),comm_set_final)]
    print("CDIM:",R_seed)
    R_seed = sum(R_seed, [])
    CDIMresult=findcommunity(G,R_seed,k, comm_set_final,'DCDIM')
    

    print('\n\n--------------- FIMMOGA -------------------')
    st=time.time()
    FIMMOGA_seed = runFIMMOGA(G,k,comm_set_final)
    FIMMOGAtime=time.time()-st
    print("FIMMOGA seed:",FIMMOGA_seed)
    FIMMOGAresult=findcommunity(G,FIMMOGA_seed,k, comm_set_final,'FIMMOGA')


    print('\n\n--------------- SSR -------------------')
    st=time.time()
    SSR_seed = runSSR(G,int(k))
    SSRtime=time.time()-st
    SSR_seed = list(SSR_seed)
    print("SSR seed:",SSR_seed)
    SSRresult=findcommunity(G,SSR_seed,k, comm_set_final,'SSR-PEA')
    
    
    
    print('\n\n--------------- CSO-IM -------------------')
    st=time.time()
    G_new=G.copy()
    CSO_SeedSet = CSOrun(G, k, comm_set_final, [], G_new)
    endtime=time.time()-st
    print("CSOseed set:",CSO_SeedSet)
    CSOResult=findcommunity(G,CSO_SeedSet,k, comm_set_final,'CSO')


#     Greedy
#     st=time.time()
#     GreedyseedSet =Greedy(G,int(k))
#     Greedytime=time.time()-st
#     print('\n\n--------------- Greedy -------------------')
#     Greedyresult=findcommunity(G,GreedyseedSet,k, comm_set_final,'Greedy')
    
    
    
#     #Single Degree Discount---Not Defined
#     st=time.time()
#     SingleDegreeSeedset = single_degree_discount(G,int(k))
#     SingleDegreetime=time.time()-st
#     print('\n\n--------------- Single Degree Discount -------------------')
#     SingleDegreeresult=findcommunity(G,SingleDegreeSeedset,k, comm_set_final,'SingleDegreeDiscount')
    
    
#     CELF
    st=time.time()
    CELFseedSet = celf(G,int(k))
    CELFtime=time.time()-st
    print('\n\n--------------- CELF -------------------')
    CELFresult=findcommunity(G,CELFseedSet,k, comm_set_final,'CELF')
    
    
    
#     PMIA
#     st=time.time()
#     PMIAseedSet = run_PMIA(G,int(k))
#     PMIAtime=time.time()-st
#     print('\n\n--------------- PMIA -------------------')
#     PMIAresult=findcommunity(G,PMIAseedSet,k, comm_set_final,'PMIA')
    
    
    
#     #SIMPATH
#     st=time.time()
#     SIMPATHseedSet = run_SIMPATH(G,int(k))
#     SIMPATHtime=time.time()-st
#     print('\n\n--------------- SIMPATH -------------------')
#     SIMPATHresult=findcommunity(G,SIMPATHseedSet,k, comm_set_final,'SIMPATH')
    
    
    
#     PAGERANK
    st=time.time()
    pagerank = sorted(nx.pagerank(G).items(), key=lambda x: x[1], reverse=True)
    pagerank_seed = [node for node, value in pagerank[0:k]]
    PRtime=time.time()-st    
    print('\n\n--------------- PAGERANK -------------------')
    Pagerankresult=findcommunity(G,pagerank_seed,k, comm_set_final,'PageRank')

#     DEGREE
    st=time.time()
    degree = sorted(G.degree(), key=lambda x: x[1], reverse=True)
    degree_seed = [node for node, value in degree[0:k]]
    Dtime=time.time()-st
    print('\n\n--------------- DEGREE -------------------')
    Degreeresult=findcommunity(G,degree_seed,k, comm_set_final,'Degree')


#     #HUB
#     st=time.time()
#     hub, authority = nx.hits(G)     
#     hub  = sorted(hub.items(), key=lambda x: x[1], reverse=True)
#     hub_seed = [node for node, value in hub[0:k]]
#     Hubtime=time.time()-st
#     print('\n\n--------------- HUB -------------------')
#     Hubresult=findcommunity(G,hub_seed,k, comm_set_final,'Hub')

#     #AUTHORITY
#     st=time.time()
#     authority  = sorted(authority.items(), key=lambda x: x[1], reverse=True)
#     authority_seed = [node for node, value in authority[0:k]]
#     Atime=time.time()-st
#     print('\n\n--------------- AUTHORITY -------------------')
#     Authorityresult=findcommunity(G,authority_seed,k, comm_set_final,'Authority')

#     #NBKCORE
#     st=time.time()
#     # neighborhood coreness
#     node2nbcore =  {node: np.sum([G.degree(nb) for nb in G.neighbors(node)]) for node in G.nodes() }
#     nbkcore = sorted(node2nbcore.items(), key=lambda x: x[1], reverse=True)
#     nbkcore_seed = [node for node, value in nbkcore[0:k]]
#     N2time=time.time()-st
#     print('\n\n--------------- neighborhood coreness -------------------')
#     Neighbourresult=findcommunity(G,nbkcore_seed,k, comm_set_final,'Neighbourhood')
    

#     resultt=[DDR,Greedyresult,CELFresult,Pagerankresult,Degreeresult,Hubresult,Authorityresult,Neighbourresult]
#     resultt=[DDR,SingleDegreeresult,CELFresult,CELFPPresult,Pagerankresult,Degreeresult,Hubresult,Authorityresult,Neighbourresult]
#     resultt=[DDR,Greedyresult,CELFresult,Pagerankresult,Degreeresult,Hubresult,Authorityresult,Neighbourresult]

#     resultt=[TBCDresult,Greedyresult,CELFresult,Pagerankresult,PMIAresult,SIMPATHresult,Degreeresult]
#     resultt=[TBCDresult,CELFresult,Pagerankresult,PMIAresult,SIMPATHresult,Degreeresult]
    resultt=[CDIMresult,FIMMOGAresult,SSRresult,Degreeresult,Pagerankresult,CELFresult,CSOResult]

    return resultt


In [24]:
def makeExcel(result,itr,perc):
    timeITR=[]
    Name=[]
    activatednodesITR=[]
    for i in result:
        for j in range(len(result[i])):
            Name.append(result[i][j]['Name'])
        break
    for i in result:
        active=[]
        timee=[]
        for j in range(len(result[i])):
            active.append(result[i][j]['length of activated nodes'])
            timee.append(result[i][j]['number of communities'])
        activatednodesITR.append(active)
        timeITR.append(timee)
    df=pd.DataFrame()
    time="Community"
    for i in range(len(timeITR)):
        timestr=time+"_"+str(itr[i])+"_"+str(perc[i])+"%"
        df[timestr]=timeITR[i]
#     print("After name",df)
    ICnodes="Activated_Nodes"
    for i in range(len(activatednodesITR)):
        ICnodesstr=ICnodes+"_"+str(itr[i])+"_"+str(perc[i])+"%"
        df[ICnodesstr]=activatednodesITR[i]
    df.insert(0,"Name of Algorithm",Name)
    return df

# upperBound_dict = {
#         'Name':algoname,
#         'k_nodes': k,
#         'Total communitites':len(comm),
#         'number of communities':noofcommunity,
#         'length of activated nodes': len(activated),
        
#         # 'length of communities': len(community_df['Unnamed: 1'].unique())
#     }
# result_df=pd.DataFrame()
# for i in range(1):
#     obj=TBCD_mine()
#     perc=[10,20,30,40,50,60,70,80,90,100]
#     filename='sample'
#     dff=obj.execute_TBCD_txt(filename,perc)
#     dff2=result_df
#     result_df=dff
#     result_df = dff2.append(dff,ignore_index = True)
#     result_df.to_excel('Outputt.xlsx')

In [25]:
class TBCD_mine(object):
    """
        tree based community detection
    """

    def __init__(self, filename=None, g=nx.Graph(), ttl=float('inf'), obs=7, path="",
                 start=None, end=None, level_max = 6, window_size = 20, theta = 0.5,
                 dataset = None, edge_list = None, comm_list = None,window_ratio = 0.10):
        """
            Constructor
            :param g: networkx graph
            :param ttl: edge time to live (days)
            :param obs: observation window (days)
            :param path: Path where generate the results and find the edge file
            :param start: starting date
            :param end: ending date
        """
        print("initialization")
        self.path = path
        self.graph = g
        self.removed = 0
        self.added = 0
        self.filename = filename
        self.start = start
        self.end = end
        self.obs = obs
        self.communities = {}
        self.intra_conn = {}
        self.inter_conn = {}
        self.select = {}
        self.cluster_head = set()
        self.cluster_h = {}
        self.level = {}
        self.occurence = {}
        self.level_max = level_max
        self.w_temp = 0
        self.window_size = window_size
        self.theta = theta
        self.dataset = ""
        self.edge_list = ""
        self.comm_list = ""
        self.ground_truth_comm = {}
        self.precision = -1
        self.recall = -1
        self.nmi = -1
        self.f_measure = -1
        self.purity = -1
        self.ari = -1
        self.entropy = -1
        self.modularity = -1
        self.coverage = -1
        self.external_density = -1
        self.average_isolability = -1
        self.mat_file_adj = ""
        self.mat_file_label = ""
        self.row = 1
        self.result_array = []
        self.window_ratio = window_ratio


    def detachability(self,label):

#         print("calculating detachability for label = "+str(label))
        G = self.graph
        internal = 0
        external = 0
        DZ = 0
        '''cluster_h_set = set()
        cluster_head_set = set()
        for node in G :
            cluster_h_set.add(self.cluster_h[node])
            cluster_head_set.add(self.cluster_head[node])
        count_h = len(cluster_h_set)
        count_head = len(cluster_head_set)
        print("\n\n\n count h = "+str(count_h))
        print("\n\n\n count head = " + str(count_head))'''
        # node and node neighbour only taken into account
        for node in G:
            if self.cluster_h[node] == label:
                for node_neighbour in G.neighbors(node):
                    if self.cluster_h[node_neighbour] == label:
                        internal = internal + G[node][node_neighbour]['weight']
                    else:
                        external = external + G[node][node_neighbour]['weight']
                '''internal += self.intra_conn[node]
                external += self.inter_conn[node]'''
        if internal + external != 0:
            DZ = internal / (internal + external)
#         print("detachbility = "+str(DZ))
        return DZ

    def max_comm_label(self,node):
        #removed influence part
        G = self.graph
        all_labels = set()
        # print("initially for node "+str(node)+" label is "+str(var_dict[node]))
        for node_neighbour in G.neighbors(node):
            all_labels.add(self.cluster_h[node_neighbour])
        prob_actual = 1
        label_actual = self.cluster_h[node]
        for label in all_labels:
            # print("for label "+str(label))
            '''prob_new = 1
            for node_chk in G.neighbors(node):
                # print("u is-"+str(u)+" v is-"+str(v))
                if self.cluster_h[node_chk] == label:
                    # print("prob_new = "+str(prob_new)+" edge weight "+str(G[node][node_chk]['weight']))
                    chk = 0
                    if G.has_edge(node, node_chk):
                        chk = G[node][node_chk]['weight']
                    if var_dict['influence'][node][node_chk] == 1:
                        # print("influence and edge weight true for "+str(node)+"-"+str(node_chk))
                        prob_new = prob_new * (1 - chk)
            if prob_new < prob_actual:
                prob_actual = prob_new
                label_actual = label
                self.cluster_h[node] = label'''
        # print("after max_comm_label for node " + str(node) + " label is " + str(var_dict[node]))
        return label_actual

    def isolability_measure_single_label(self,label):

        G = self.graph
        isolability = 0
        internal = 0
        external = 0
        for node in G:
            if self.cluster_h[node] == label:
                for node_neighbour in G.neighbors(node):
                    if (self.cluster_h[node] == self.cluster_h[node_neighbour]):
                        internal = internal + G[node][node_neighbour]['weight']
                    else:
                        external = external + G[node][node_neighbour]['weight']
        if external != 0: isolability = internal / (internal+external)
        return isolability

    def external_density_eval(self):

        G = self.graph
        numerator = 0
        n = len(G)
        denominator = n * (n - 1)
        total_labels = set()
        for node in G:
            total_labels.add(self.cluster_h[node])
        for label in total_labels:
            nodes_per_label = 0
            for node in G:
                if self.cluster_h[node] == label: nodes_per_label += 1
            denominator -= (nodes_per_label * (nodes_per_label - 1))
        for (node1, node2) in G.edges():
            if self.cluster_h[node1] != self.cluster_h[node2] and node1 != node2: numerator += 1
        if denominator != 0:
            return numerator / denominator
        else:
            return 0

    def coverage_eval(self):

        G = self.graph
        numerator = 0
        denominator = len(G.edges)
        for (node1, node2) in G.edges():
            if self.cluster_h[node1] == self.cluster_h[node2] and node1 != node2: numerator += 1
        if denominator != 0:
            return numerator / denominator
        else:
            return 0

    def modularity_eval(self):

        G = self.graph
        total_edges = len(G.edges)
        total_labels = set()
        for node in G:
            total_labels.add(self.cluster_h[node])
        modularity = 0
        internal_final = 0
        external_final = 0
        for label in total_labels:
            internal = 0
            external = 0
            for node in G:
                if self.cluster_h[node] == label:
                    for node_neighbour in G.neighbors(node):
                        if self.cluster_h[node_neighbour] == label:
                            internal += 1
                        else:
                            external += 1
            '''internal_final_check = internal/total_edges
            external_final_check = external/total_edges
            if internal_final_check >= 1 : print("internal problem")
            if external_final_check >= 1: print("external problem")
            modularity_current_label = internal_final - external_final*external_final
            if modularity_current_label >= 1 : print("modularity current label problem")
            modularity += ((internal/total_edges) - ((external*external)/(total_edges*total_edges)))'''
            modularity += internal / len(G.edges) - ((external * external) / (len(G.edges) * len(G.edges)))
            internal_final += internal
            external_final += external
        internal_final = internal_final / 2
        external_final = external_final / 2
        modularity_final = (internal_final / total_edges) - (
                    (external_final * external_final) / (total_edges * total_edges))
        modularity_final = modularity / 2
        if (internal_final + external_final) == total_edges:
            print("modularity edge check correct")
            print(str(modularity_final))
        return modularity_final

    def TBCD_dissol(self):

        G = self.graph
        print("Inside dissolution phase")
        labels = set()
        for node in self.cluster_head: labels.add(node) 
#         print("labels:",labels)
        
#         print("no of sets = " + str(len(labels)))
#         print("no of nodes = " + str(len(G)))
        for label in labels:
            comm_set = []
            for node in self.graph:
                if self.cluster_h[node] == label:
                    comm_set.append(node)
                    comm_set.append(self.level[node])
#                     print("Community:",comm_set)
                    
        label_set = set()
        for label in self.cluster_head:
            label_set.add(label)
#         print("label_set:",label_set)
        
        for label in label_set :
#             print("for label = "+str(label))
            detachabil = self.detachability(label)
#             print("detachabil:",detachabil,"theta:",self.theta)
            
            if detachabil < self.theta :
                n_e = set()
                for node in G :
                    self.occurence[node] = 0
                for node in G :
                    if self.cluster_h[node] == label :
                        for single in G.neighbors(node) :
                            if self.cluster_h[single] != label :
                                n_e.add(single)
                                self.occurence[single] += 1
#                 print("n_e set has = "+str(len(n_e)))
                max = -1
                for node in G :
                    if self.occurence[node] > max :
                        max = self.occurence[node]
                n_max = set()
                for node in G :
                    if self.occurence[node] == max :
                        n_max.add(node)
#                 print("n_max set has = " + str(len(n_max)))
                
                c_s = set()
                for node in n_max :
                    c_s.add(self.cluster_h[node])
#                 print("c_s set has = " + str(len(c_s)))
                mid = -999999
                big_label = -1
                for other_label in c_s :
                    label_node_set = set()
                    for node in G :
                        if self.cluster_h[node] == label :
                            label_node_set.add(node)
                    for node in label_node_set :
                        self.cluster_h[node] = other_label
                    term1 = self.detachability(other_label)
                    for node in label_node_set :
                        self.cluster_h[node] = label
                    term2 = self.detachability(other_label)
                    tid = term1 - term2
#                     print("for label = "+str(other_label))
#                     print("tid = "+str(tid))
#                     print("mid = " + str(mid))
                    if tid > mid :
#                         print("big label is other label")
                        mid = tid
                        big_label = other_label
                to_be_removed = -9999
                if len(self.cluster_head) != 1 :
                    self.cluster_head.remove(label)
                    to_be_removed = label
#                     print("big label = "+str(big_label))
#                     print("to be removed label = " + str(label))
                    for node in G :
                        if self.cluster_h[node] == to_be_removed :
                            self.cluster_h[node] = big_label

    def combin(self,n):

        n = int(n)
        term = n*(n-1)
        if term > 2:
            return term/2
        else : return 0


    def all_metric_nog(self):

        G = self.graph

        self.modularity = self.modularity_eval()
        self.coverage = self.coverage_eval()
        self.external_density = self.external_density_eval()
        print("\n\n\n\nwithout ground truth\n\n\n\n")
        print("modularity = " + str(self.modularity))
        print("coverage = " + str(self.coverage))
        print("external density = " + str(self.external_density))
        total_labels_mine = set()
        for node in G:
            total_labels_mine.add(self.cluster_h[node])
        total_isolability = 0
        for label in total_labels_mine:
            total_isolability = total_isolability + self.isolability_measure_single_label(label)
        average_isolability = total_isolability / len(total_labels_mine)
        self.average_isolability = average_isolability
        print("average isolability = " + str(average_isolability))

        self.result_array.append([self.row,len(total_labels_mine),self.modularity,self.coverage,self.external_density,
                                  self.average_isolability])
        self.row += 1

    def all_metric(self):

        G = self.graph

        self.modularity = self.modularity_eval()
        self.coverage = self.coverage_eval()
        self.external_density = self.external_density_eval()
        print("\n\n\n\nwithout ground truth\n\n\n\n")
        print("modularity = " + str(self.modularity))
        print("coverage = " + str(self.coverage))
        print("external density = " + str(self.external_density))
        total_labels_mine = set()
        total_labels_ground = set()
        for node in G:
            total_labels_mine.add(self.cluster_h[node])
            #total_labels_ground.add(self.ground_truth_comm[node])
        total_isolability = 0
        for label in total_labels_mine:
            total_isolability = total_isolability + self.isolability_measure_single_label(label)
        average_isolability = total_isolability / len(total_labels_mine)
        self.average_isolability = average_isolability
        print("average isolability = " + str(average_isolability))

        print("calculating ground truth metric")
        comm_list = open(self.comm_list)
        print("before reading community text file")

        no_nodes = 0
        ground_truth_comm_set = set()
        for l in comm_list:
            l = l.split(" ")
            node = int(l[0])
            comm_label = int(l[1])
            #print("for node x : " + str(node) + " community is  : " + str(comm_label))
            ground_truth_comm_set.add(comm_label)
            self.ground_truth_comm[node] = comm_label
            no_nodes += 1

        print("no of communities in ground truth = "+str(len(ground_truth_comm_set)))
        #print(str(ground_truth_comm_set))
        #for node in range(no_nodes): print(str(node)+" "+str(self.ground_truth_comm[node]))

        algo_comm_set = set()
        for node in G :
            algo_comm_set.add(self.cluster_h[node])
        print("no of communities from algo = "+str(len(algo_comm_set)))


        ground_truth_comm_list = list(ground_truth_comm_set)
        algo_comm_list = list(algo_comm_set)
        common_matrix = np.zeros((len(ground_truth_comm_set),len(algo_comm_set)))
        for i in range(len(ground_truth_comm_set)) :
            for j in range(len(algo_comm_set)) :
                for node in G :
                    if self.ground_truth_comm[node] == ground_truth_comm_list[i] and \
                            self.cluster_h[node] == algo_comm_list[j] :
                        common_matrix[i][j] += 1
        true_pos = 0
        false_pos = 0
        true_neg = 0
        false_neg = 0
        for node1 in G :
            for node2 in G :
                if self.cluster_h[node1] == self.cluster_h[node2] :
                    if self.ground_truth_comm[node1] == self.ground_truth_comm[node2] :
                        true_pos += 1
                    else :
                        false_pos += 1
                else :
                    if self.ground_truth_comm[node1] != self.ground_truth_comm[node2] :
                        true_neg += 1
                    else :
                        false_neg += 1

        self.precision = true_pos/(true_pos+false_pos)
        self.recall = true_pos/(true_pos+false_neg)

        ground_truth_comm_no = {}
        ground_truth_comm_no_index = np.zeros(len(ground_truth_comm_set))
        algo_comm_no = {}
        algo_comm_no_index = np.zeros(len(algo_comm_set))
        index = -1
        for label in ground_truth_comm_set :
            index += 1
            ground_truth_comm_no[label] = 0
            ground_truth_comm_no_index[index] = 0
            for node in range(no_nodes) :
                if self.ground_truth_comm[node] == label :
                    #print("counting ground truth increase")
                    ground_truth_comm_no[label] += 1
                    ground_truth_comm_no_index[index] += 1

        index = -1
        for label in algo_comm_set:
            index += 1
            algo_comm_no[label] = 0
            algo_comm_no_index[index] = 0
            for node in G:
                if self.cluster_h[node] == label:
                    algo_comm_no[label] += 1
                    algo_comm_no_index[index] += 1

        numerator = 0
        denominator = 0

        for i in range(len(ground_truth_comm_set)):
            if ground_truth_comm_no_index[i] == 0 : print("error in ground truth label counting")


        for i in range(len(ground_truth_comm_set)) :
            for j in range(len(algo_comm_set)) :
                if common_matrix[i][j] != 0 :
                    numerator += common_matrix[i][j] * math.log((common_matrix[i][j]*no_nodes)/(algo_comm_no_index[j]*ground_truth_comm_no_index[i]))

        numerator *= -2

        for i in range(len(ground_truth_comm_set)):
            denominator += ground_truth_comm_no_index[i] * math.log(ground_truth_comm_no_index[i]/no_nodes)

        for j in range(len(algo_comm_set)):
            denominator += algo_comm_no_index[j] * math.log(algo_comm_no_index[j]/no_nodes)

        self.nmi = numerator/denominator

        self.f_measure = 2 * (self.precision * self.recall)/(self.precision + self.recall)

        self.purity = 0

        for ground_truth_label in ground_truth_comm_set:
            max = -1
            for algo_label in algo_comm_set:
                count = 0
                for node in G :
                    if self.cluster_h[node] == algo_label and self.ground_truth_comm[node] == ground_truth_label :
                        count += 1
                if count > max : max = count

            self.purity += max

        self.purity = self.purity/no_nodes

        self.entropy = 0

        for label_mine in algo_comm_set :
            n_c = algo_comm_no[label_mine]
            n = no_nodes
            m = len(ground_truth_comm_set)
            term2 = 0
            for label_ground in ground_truth_comm_set :
                n_i_j = 0
                for node in G :
                    if self.cluster_h[node] == label_mine and self.ground_truth_comm[node] == label_ground :
                        n_i_j += 1
                if n_i_j != 0 :
                    term2 += (n_i_j/n_c)* math.log(n_i_j/n_c)
            self.entropy += (n_c/n)*(1/math.log(m))*term2*(-1)

        print("entropy = " + str(self.entropy))

        self.ari = 0
        term_common = 0
        for i in range(len(ground_truth_comm_set)):
            for j in range(len(algo_comm_set)):
                term_common += self.combin(common_matrix[i][j])

        term_ground = 0

        print("ground truth label set")
        print(str(ground_truth_comm_set))
        for label in ground_truth_comm_set:
            count = 0
            for node in G:
                if self.ground_truth_comm[node] == label: count += 1
            term_ground += self.combin(count)

        print("algo label set")
        print(str(algo_comm_set))
        term_algo = 0
        for label in algo_comm_set:
            count = 0
            for node in G:
                if self.cluster_h[node] == label: count += 1
            term_algo += self.combin(count)


        term_sum = (term_ground + term_algo) / 2
        term_prod = (term_ground * term_algo) / self.combin(no_nodes)

        '''print("term_common = "+str(term_common))
        print("term_ground = " + str(term_ground))
        print("term_algo = " + str(term_algo))
        print("term_sum = " + str(term_sum))
        print("term_prod = " + str(term_prod))'''

        self.ari = (term_common - term_prod) / (term_sum - term_prod)

        self.result_array.append([self.row,len(algo_comm_set),self.modularity,self.coverage,self.external_density,self.average_isolability,self.f_measure,self.nmi,self.purity,self.entropy,self.ari])
        self.row += 1
        
    def calldiverified(self):
        print(self.graph)
        return 1,3
    

    def execute_TBCD_txt(self,file_name,perc):
        """
            Execute tree based community detection algorithm from static dataset
        """

        # print("reading static edge list with file name : "+str(file_name))
#         self.edge_list = './datasets_txt/' + file_name + '_edge_list.txt'
#         self.comm_list = './datasets_txt/' + file_name + '_comm_list.txt'
        self.edge_list = file_name + '_edge_list.txt'
#         self.comm_list =  file_name + '_comm_list.txt'
    
        edge_list = open(self.edge_list)
#         print("counting edges to set window size")
        window_counter = 0
        count = 0
        for l in edge_list:
            count += 1
#         print("Count:",count,"Window ratio:",self.window_ratio)
        
        self.window_size = int(count*self.window_ratio)
        # print("Window size:",self.window_size)
        
        self.graph = nx.Graph()
        # print(self.graph)
        
#       print("before reading edge list text file")
        no_nodes = 0
        w = 0
        edge_list = open(self.edge_list)
        R_seed=[]
        seedSet=[]
        pagerank=[]
        G = nx.DiGraph()
#         MultiDiGraph
        # print("------------------------------------------------------------------------------------------")
        
        print(count)
        itr=[]
        for i in range(len(perc)):
            vv=int((perc[i]/100)*count)
            itr.append(vv)
        print(itr)
        totalnodes=0
        resultt={}
        
        for l in edge_list:
#             print(l)
            totalnodes=totalnodes+1
            l = l.split(" ")
            s=int(l[0])
            t=int(l[1])
            value = round(random.uniform(0.01,1.0),2)
            G.add_edge(s,t,weight=value)
            G.nodes[s]['thres']=(G.in_degree(s)/2)
            G.nodes[t]['thres']=(G.in_degree(t)/2)
            k=int(findk(len(G.nodes)))
            
            self.added += 1
            e = {}
            x = int(l[0])
            y = int(l[1])
#             print("for edge x : "+str(x)+" y : "+str(y))
#             print(".....................................................")
            if x == y:
#                 print("self edge found")
                if totalnodes in itr:
                    self.TBCD_dissol()
                    w = 0
                    print("_______________________________________________________________________________________________")
                    print("Get result at edge:",totalnodes)
                    comm_set_final = detect_comm(self.graph, self.cluster_head, self.cluster_h)
                    resultt[totalnodes]=findresult(G,k,comm_set_final)
                continue

            if not self.graph.has_node(x) :
#                 print("Graph Does not have node x.")
                self.graph.add_node(x)
#                 print("intra connection of x:",self.intra_conn[x])
                self.intra_conn[x] = -1
                self.inter_conn[x] = -1
                self.select[x] = 0
                no_nodes = no_nodes + 1
#                 print("new node added")
                # print("no nodes = "+str(no_nodes))

            if not self.graph.has_node(y):
#                 print("Graph Does not have node y.")
                self.graph.add_node(y)
#                 print("intra connection of y:",self.intra_conn[y])
                self.intra_conn[y] = -1
                self.inter_conn[y] = -1
                self.select[y] = 0
                no_nodes = no_nodes + 1
#                 print("new node added")
                # print("no nodes = " + str(no_nodes))

            self.graph.add_edge(x, y, weight=1.0)

            if self.select[x] == 0 and self.select[y] == 0:
#                 print("If both are new.")
#                 print("select[x]==select[y]")
                # print(type(self.cluster_head))
#                 print("Cluster head:",self.cluster_head)
                self.cluster_head.add(x)
#                 print("Cluster head after added:",self.cluster_head)
                self.cluster_h[x] = x
                self.level[x] = 0
                self.level[y] = 1
                self.cluster_h[y] = x
                self.intra_conn[x] = 1
                self.intra_conn[y] = 1
                self.select[x] = 1
                self.select[y] = 1

            elif self.select[y] == 0 or self.select[x] == 0 :

                if self.select[x] == 0 :
                    term = y
                    y = x
                    x = term
#                 print("Max level:",self.level_max)
                
                
                if self.level[x] <= self.level_max - 1:
                    self.level[y] = self.level[x] + 1
                    self.intra_conn[x] += 1
                    self.intra_conn[y] = 1
                    self.cluster_h[y] = self.cluster_h[x]
                    self.select[y] = 1

                else:
                    self.cluster_head.add(x)
                    self.cluster_h[x] = x
                    self.level[x] = 0
                    self.level[y] = 1
                    self.cluster_h[y] = x
                    self.inter_conn[x] = self.intra_conn[x]
                    self.intra_conn[x] = 1
                    self.intra_conn[y] = 1
                    self.select[x] = 1
                    self.select[y] = 1

            else:  #If clusters are same
#                 print("If clusters are same...,")
                if self.cluster_h[x] == self.cluster_h[y]:
                    self.intra_conn[x] += 1
                    self.intra_conn[y] += 1
                else:
                    self.inter_conn[x] += 1
                    self.inter_conn[y] += 1
#             print("cluster_h x:",self.cluster_h[x],"cluster_h y:",self.cluster_h[y])
#             print("Expansion phase started.")
            
#             labels = set()
#             for node in self.cluster_head: labels.add(node)
#             comm_set_final = []
#             for label in labels:
#                 comm_set = []
#                 for node in self.graph :
#                     if self.cluster_h[node] == label :
#                         comm_set.append(node)
#                 comm_set_final.append(comm_set)
            w += 1
            if w == self.window_size or totalnodes==count:
                print("**************************************************************************")
                window_counter += 1  
                self.TBCD_dissol()
                w = 0
            if totalnodes in itr:
                self.TBCD_dissol()
                w = 0
                print("_______________________________________________________________________________________________")
                print("Get result at node:",totalnodes)
                comm_set_final = detect_comm(self.graph, self.cluster_head, self.cluster_h)
                resultt[totalnodes]=findresult(G,k,comm_set_final)
        
        displayresult(resultt,itr,perc)
        dff = makeExcel(resultt,itr,perc)
        return dff

            
    def checkingg(self):
        print("666")
        
        
def detect_comm(graph, cluster_head, cluster_h):
    labels = set()
    for node in cluster_head:
        labels.add(node)
    comm_set_final = []
    for label in labels:
        comm_set = []
        for node in graph :
            if cluster_h[node] == label :
                comm_set.append(node)
        comm_set_final.append(comm_set)
    return comm_set_final
# obj=TBCD_mine()
# perc=[20,40,60,80,100]
# filename='LFR_500_0.0'
# obj.execute_TBCD_txt(filename,perc)

In [26]:
# obj=TBCD_mine()
# perc=[20,40,60,80,100]
# filename='LFR_500_0.0'
# obj.execute_TBCD_txt(filename,perc)

In [27]:
print("hello.")

hello.


In [ ]:
def executeDiv():
    result_df=pd.DataFrame()
    for i in range(1):
        obj=TBCD_mine()
#         perc=[10,20,30,40,50,60,70,80,90,100]
        perc=[80,90,100]
        filename='cond_mat2001'
        dff=obj.execute_TBCD_txt(filename,perc)
        dff2=result_df
        result_df=dff
        result_df = pd.concat([dff2, dff], ignore_index=True)
        result_df.to_excel('Outputt.xlsx')
def processResults():
    df = pd.read_excel('Outputt.xlsx')
    df=df.drop(columns='Unnamed: 0')
    print(df.shape)
#     print(df.head(8))
    TBCD = df.loc[df['Name of Algorithm']=='TBCD']
    DCDIM = df.loc[df['Name of Algorithm']=='DCDIM']
    Greedy = df.loc[df['Name of Algorithm']=='Greedy']
    CELF = df.loc[df['Name of Algorithm']=='CELF']
    PMIA = df.loc[df['Name of Algorithm']=='PMIA']
    FIMMOGA = df.loc[df['Name of Algorithm']=='FIMMOGA']
    cso = df.loc[df['Name of Algorithm']=='CSO']
    SSRPEA = df.loc[df['Name of Algorithm']=='SSR-PEA']
    SIMPATH = df.loc[df['Name of Algorithm']=='SIMPATH']
    DEGREE = df.loc[df['Name of Algorithm']=='Degree']
    PAGERANK = df.loc[df['Name of Algorithm']=='PageRank']
    
    cols= df.columns

    ddris = pd.DataFrame(TBCD.iloc[:,1:].mean(),columns = ['TBCD'])
    dcdim = pd.DataFrame(DCDIM.iloc[:,1:].mean(),columns = ['DCDIM'])
    greedy = pd.DataFrame(Greedy.iloc[:,1:].mean(),columns = ['Greedy'])
    celf = pd.DataFrame(CELF.iloc[:,1:].mean(),columns = ['CELF'])
    pmia = pd.DataFrame(PMIA.iloc[:,1:].mean(),columns = ['PMIA'])
    simpath = pd.DataFrame(SIMPATH.iloc[:,1:].mean(),columns = ['SIMPATH'])
    degree = pd.DataFrame(DEGREE.iloc[:,1:].mean(),columns = ['DEGREE'])
    pagerank = pd.DataFrame(PAGERANK.iloc[:,1:].mean(),columns = ['PageRank'])
    CSO = pd.DataFrame(cso.iloc[:,1:].mean(),columns = ['CSO'])
    FIMMOGA = pd.DataFrame(FIMMOGA.iloc[:,1:].mean(),columns = ['FIMMOGA'])
    SSRPEA = pd.DataFrame(SSRPEA.iloc[:,1:].mean(),columns = ['SSRPEA'])
    
    
    resultant = pd.concat([dcdim,celf,pmia,degree,pagerank,CSO, FIMMOGA,SSRPEA], axis='columns')
#     resultant = pd.concat([ddris,pmia,simpath,degree,pagerank], axis='columns')
#     print("Resultant:",resultant)
    
    result=resultant.iloc[0:]
#     print("Result:",result)
    
    result.to_excel('cond_mat2001_l05s003_Processed.xlsx')
    
    
executeDiv()
processResults()

initialization
47594
[38075, 42834, 47594]
**************************************************************************
Inside dissolution phase
**************************************************************************
Inside dissolution phase
**************************************************************************
Inside dissolution phase
**************************************************************************
Inside dissolution phase
**************************************************************************
Inside dissolution phase
**************************************************************************
Inside dissolution phase
**************************************************************************
Inside dissolution phase
**************************************************************************
Inside dissolution phase
Inside dissolution phase
_______________________________________________________________________________________________
Get result at node: 38075


--------